In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install -U -q "tf-models-official"
!pip install -U nvidia-cudnn-cu11

# Install the mediapy package for visualizing images/videos.
# See https://github.com/google/mediapy
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy remotezip
!pip install -U -q git+https://github.com/tensorflow/docs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 93.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts

In [3]:
!pip install tf-models-official  --quiet
!pip install tensorflow-docs --quiet
!pip install -U kaleido --quiet
!pip install tf-keras-vis --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.1 MB/s eta 0:00:00


In [4]:
import random
import pathlib
import itertools
import collections
from collections import defaultdict

import cv2
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import imageio
import gc

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, TensorBoard, LearningRateScheduler

In [5]:
from official.projects.movinet.modeling import movinet
from official.projects.movinet.modeling import movinet_model
from official.projects.movinet.tools import export_saved_model

In [6]:
def classes_available(path):
    return [c for c in os.listdir(path)]

def files_per_class(path):
    class_instances = dict()
    for c in classes_available(path):
        class_path = os.path.join(path,c)
        files = list()
        for file in os.listdir(class_path):
            files.append(os.path.join(class_path,file))
        class_instances[c.split('-')[0]] = files

    return class_instances

t = files_per_class('/kaggle/input/footballactions/FootballActions')

print('Num classes:', len(list(t.keys())))
for i in t.keys():
    print(f'Num videos for class {i}:' ,len(t[i]))


def average_frames_and_length_per_class(path):
    class_instances = files_per_class(path)
    avg_frames_per_class = {}
    avg_length_per_class = {}

    for class_name, files in class_instances.items():
        total_frames = 0
        total_length = 0
        for file in files:
            cap = cv2.VideoCapture(file)
            frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            fps = cap.get(cv2.CAP_PROP_FPS)
            duration = frame_count / fps if fps > 0 else 0

            total_frames += frame_count
            total_length += duration
            cap.release()

        num_files = len(files)
        avg_frames = total_frames / num_files if num_files else 0
        avg_length = total_length / num_files if num_files else 0

        avg_frames_per_class[class_name] = avg_frames
        avg_length_per_class[class_name] = avg_length

    return avg_frames_per_class, avg_length_per_class


avg_frames_per_class, avg_length_per_class = average_frames_and_length_per_class('/kaggle/input/footballactions/FootballActions')

print('\nAverage frames and length per class:')
for class_name in avg_frames_per_class.keys():
    print(f'{class_name}:')
    print(f'  Avg frames: {avg_frames_per_class[class_name]:.2f}')
    print(f'  Avg length: {avg_length_per_class[class_name]:.2f} seconds')

def train_test_split(dataset_dict, test_split=0.2, seed=42):
    random.seed(seed)
    train_dict = dict()
    test_dict = dict()

    for class_name, files in dataset_dict.items():
        random.shuffle(files)
        split_index = int(len(files) * (1 - test_split))
        train_files = files[:split_index]
        test_files = files[split_index:]
        train_dict[class_name] = train_files
        test_dict[class_name] = test_files

    return train_dict, test_dict

Num classes: 8
Num videos for class shortpass: 354
Num videos for class longpass: 147
Num videos for class throw: 367
Num videos for class goalkick: 425
Num videos for class penalty: 322
Num videos for class corner: 411
Num videos for class freekick: 374
Num videos for class ontarget: 340

Average frames and length per class:
shortpass:
  Avg frames: 75.86
  Avg length: 2.99 seconds
longpass:
  Avg frames: 101.80
  Avg length: 4.07 seconds
throw:
  Avg frames: 62.64
  Avg length: 2.46 seconds
goalkick:
  Avg frames: 73.02
  Avg length: 2.88 seconds
penalty:
  Avg frames: 71.91
  Avg length: 2.86 seconds
corner:
  Avg frames: 75.38
  Avg length: 2.89 seconds
freekick:
  Avg frames: 74.21
  Avg length: 2.88 seconds
ontarget:
  Avg frames: 69.32
  Avg length: 2.48 seconds


In [7]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff

In [8]:
def per_model(t, train_dict, test_dict, model_class, split):
    split_percentage = 100 * split
    classes = list(t.keys())

    # Get train and test counts per class
    train_counts = [len(train_dict[c]) for c in classes]
    test_counts = [len(test_dict[c]) for c in classes]

    # Sort classes by total count (train + test)
    sorted_data = sorted(zip(classes, train_counts, test_counts), key=lambda x: x[1] + x[2], reverse=True)
    sorted_classes, sorted_train_counts, sorted_test_counts = zip(*sorted_data)

    fig = go.Figure()

    # Add train data bars with labels
    fig.add_trace(go.Bar(
        x=sorted_classes,
        y=sorted_train_counts,
        name="Train",
        marker_color="#9370DB",  # Medium purple
        text=[f"{count:,}" for count in sorted_train_counts],
        textposition="outside"  # Moves labels outside bars
    ))

    # Add test data bars with labels
    fig.add_trace(go.Bar(
        x=sorted_classes,
        y=sorted_test_counts,
        name="Test",
        marker_color="#D3D3D3",  # Light gray
        text=[f"{count:,}" for count in sorted_test_counts],
        textposition="outside"
    ))

    # Calculate overall dataset statistics
    total_train = sum(train_counts)
    total_test = sum(test_counts)
    total_samples = total_train + total_test

    # Add total annotations above bars
    for i, (cls, train, test) in enumerate(zip(sorted_classes, sorted_train_counts, sorted_test_counts)):
        total_count = train + test
        percentage = (total_count / total_samples) * 100
        fig.add_annotation(
            x=cls, y=total_count + max(total_samples * 0.02, 5),  # Offset for visibility
            text=f"{total_count:,} ({percentage:.1f}%)",
            showarrow=False,
            font=dict(size=12, color="black")
        )

    # Layout updates
    fig.update_layout(
        title=dict(
            text=f"Train-Test Split Distribution for {model_class}",
            font=dict(family="Segoe Print", size=24),
            y=0.95, x=0.5, xanchor="center", yanchor="top"
        ),
        xaxis_title=dict(text="Class", font=dict(family="Segoe Print", size=18)),
        yaxis_title=dict(text="Number of Videos", font=dict(family="Segoe Print", size=18)),
        annotations=[
            dict(
                x=0.5, y=1.1,
                text=f"Total: {total_samples:,} | Train: {total_train:,} ({total_train / total_samples:.1%}) | Test: {total_test:,} ({total_test / total_samples:.1%})",
                xref="paper", yref="paper",
                showarrow=False,
                font=dict(family="Segoe Print", size=16, color="darkred")
            )
        ],
        hovermode=False,  # No need for hover since it's a static image
        margin=dict(t=150),
        font=dict(family="Segoe Print"),
        plot_bgcolor="#F8F8F8",
        paper_bgcolor="#F8F8F8"
    )

    # X-axis formatting
    fig.update_xaxes(
        tickangle=45,
        tickfont=dict(family="Segoe Print", size=12),
        showline=True,
        linewidth=1,
        linecolor="black"
    )

    # Y-axis formatting
    fig.update_yaxes(
        gridcolor="white",
        griddash="dash"
    )

    # Save as PNG
    file_name = f"train_test_split_{model_class}_split_{100 - split_percentage:.0f}_{split_percentage:.0f}.png"
    fig.write_image(file_name, scale=3, width=1200, height=800)
    print(f"Plot saved: {file_name}")

In [9]:
def format_frames(frame, output_size):
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize_with_pad(frame, *output_size)
    return frame
    
def create_blank_frames(n_frames, output_size):
    """
    Creates blank frames for error cases.
    """
    return np.zeros((n_frames, output_size[0], output_size[1], 3), dtype=np.float32)

In [10]:
def frames_from_video_file(video_path, n_frames, output_size = (172,172), frame_step = 15):
      """
        Creates frames from each video file present for each category.
    
        Args:
          video_path: File path to the video.
          n_frames: Number of frames to be created per video file.
          output_size: Pixel size of the output frame image.
    
        Return:
          An NumPy array of frames in the shape of (n_frames, height, width, channels).
      """
      # Read each video frame by frame
      result = []
      src = cv2.VideoCapture(str(video_path))
    
      video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)
    
      need_length = 1 + (n_frames - 1) * frame_step
    
      if need_length > video_length:
        start = 0
      else:
        max_start = video_length - need_length
        start = random.randint(0, max_start + 1)
    
      src.set(cv2.CAP_PROP_POS_FRAMES, start)
      # ret is a boolean indicating whether read was successful, frame is the image itself
      ret, frame = src.read()
      result.append(format_frames(frame, output_size))  
      for _ in range(n_frames - 1):
        for _ in range(frame_step):
          ret, frame = src.read()
        if ret:
          frame = format_frames(frame, output_size)
          result.append(frame)
        else:
          result.append(np.zeros_like(result[0]))
      src.release()
      result = np.array(result)[..., [2, 1, 0]]
    
      return result

In [11]:

class FrameGenerator:
    def __init__(self, dataset_dict, num_frames, size=290, training=False):
        self.dataset_dict = dataset_dict
        self.num_frames = num_frames
        self.size = size
        self.training = training
        self.class_names = sorted(dataset_dict.keys())
        self.class_ids_for_name = dict((name, idx) for idx, name in enumerate(self.class_names))
    
    def __call__(self):
        video_paths = []
        classes = []
        for class_name, files in self.dataset_dict.items():
            video_paths.extend(files)
            classes.extend([class_name] * len(files))
        
        pairs = list(zip(video_paths, classes))
        if self.training:
            random.shuffle(pairs)
            
        for video_path, class_name in pairs:
            try:
                video_frames = frames_from_video_file(video_path, 
                                                   self.num_frames, 
                                                   (self.size, self.size))
                label = self.class_ids_for_name[class_name]
                yield video_frames, label
            except Exception as e:
                print(f"Warning: Error processing video {video_path}: {str(e)}")
                yield create_blank_frames(self.num_frames, (self.size, self.size)), 0

In [12]:
models = {
        # 'A0': {'batch_size': 10, 'num_frames': 8, 'resolution': 172,'split':0.20},
        'A3': {'batch_size': 8, 'num_frames': 8, 'resolution': 256,'split':0.30},     
}

In [13]:
def prepare_dataset(generator,batch_size):
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=(
            tf.TensorSpec(shape=(None, None, None, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(), dtype=tf.int16)
        )
    )
    return dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)

In [14]:
def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
  """Builds a classifier on top of a backbone model."""
  model = movinet_model.MovinetClassifier(
      backbone=backbone,
      num_classes=num_classes)
  model.build([batch_size, num_frames, resolution, resolution, 3])

  return model

In [15]:

class CustomLearningRateLogger(tf.keras.callbacks.Callback):
    def _implements_train_batch_hooks(self):
        return False
    def _implements_test_batch_hooks(self):
        return False
    def _implements_predict_batch_hooks(self):
        return False
    
    def on_epoch_begin(self, epoch, logs=None):
        lr = self.model.optimizer.lr
        if hasattr(lr, 'value'):
            lr = lr.value()
        tf.summary.scalar('learning_rate', data=lr, step=epoch)
        if logs is None:
            logs = {}
        logs['lr'] = lr
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = logs.get('lr', None)

class CustomModelCheckpoint(ModelCheckpoint):
    def _implements_train_batch_hooks(self):
        return False
    def _implements_test_batch_hooks(self):
        return False
    def _implements_predict_batch_hooks(self):
        return False

class CustomReduceLROnPlateau(ReduceLROnPlateau):
    def _implements_train_batch_hooks(self):
        return False
    def _implements_test_batch_hooks(self):
        return False
    def _implements_predict_batch_hooks(self):
        return False
    
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = self.model.optimizer.lr.numpy()
        super().on_epoch_end(epoch, logs)

class CustomEarlyStopping(EarlyStopping):
    def _implements_train_batch_hooks(self):
        return False
    def _implements_test_batch_hooks(self):
        return False
    def _implements_predict_batch_hooks(self):
        return False

class CustomLearningRateScheduler(LearningRateScheduler):
    def _implements_train_batch_hooks(self):
        return False
    def _implements_test_batch_hooks(self):
        return False
    def _implements_predict_batch_hooks(self):
        return False

class CustomTensorBoard(TensorBoard):
    def _implements_train_batch_hooks(self):
        return False
    def _implements_test_batch_hooks(self):
        return False
    def _implements_predict_batch_hooks(self):
        return False

In [16]:
def lr_schedule(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1))

In [17]:
from sklearn.metrics import confusion_matrix, classification_report

def get_actual_predicted_labels(dataset, model):
    actual = []
    predicted = []
    for images, labels in dataset:
        actual.extend(labels.numpy())
        preds = model.predict(images)
        predicted.extend(tf.argmax(preds, axis=1).numpy())
    return np.array(actual), np.array(predicted)

In [18]:
from matplotlib.colors import LinearSegmentedColormap

def plot_confusion_matrix(actual, predicted, labels, ds_type, model_class, accuracy, split, normalize=False):
    split = split * 100
    
    def create_cm_plot(ax, cm, cmap, title):
        sns.heatmap(cm, annot=True, fmt=fmt, cmap=cmap,
                    xticklabels=labels, yticklabels=labels,
                    annot_kws={"size": 10, "weight": "bold"}, square=True, 
                    cbar_kws={'shrink': .8}, ax=ax, linewidths=0.5, linecolor='gray')
        
        ax.set_title(title, fontsize=18, fontweight='bold', pad=20)
        ax.set_xlabel('Predicted Action', fontsize=14, fontweight='bold', labelpad=15)
        ax.set_ylabel('Actual Action', fontsize=14, fontweight='bold', labelpad=15)
        
        # Improve tick label visibility
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor", fontsize=10, fontweight='bold')
        plt.setp(ax.get_yticklabels(), rotation=0, ha="right", fontsize=10, fontweight='bold')
        
        # Add value annotations
        thresh = cm.max() / 2
        for i, j in np.ndindex(cm.shape):
            ax.text(j + 0.5, i + 0.5, f"{cm[i, j]:.0f}" if not normalize else f"{cm[i, j]:.2f}",
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black",
                    fontweight='bold', fontsize=10)

        # Add borders to the heatmap
        for _, spine in ax.spines.items():
            spine.set_visible(True)
            spine.set_linewidth(2)
            spine.set_color('black')

    try:
        actual = np.ravel(actual)
        predicted = np.ravel(predicted)
        cm = confusion_matrix(actual, predicted)
        if normalize:
            cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
            fmt = '.2f'
        else:
            fmt = 'd'
        
        fig, (ax_cm, ax_guide) = plt.subplots(1, 2, figsize=(24, 12), 
                                              gridspec_kw={'width_ratios': [1.5, 1]})
        
        # Custom colormap for better visibility
        cmap = sns.color_palette("Blues", as_cmap=True)
        create_cm_plot(ax_cm, cm, cmap, 'Confusion Matrix')
        
        # Add interpretation guide
        ax_guide.axis('off')
        guide_text = """
        Interpretation Guide:
        
        1. Diagonal Elements: 
           Represent correct predictions. Higher values are better.
        
        2. Off-Diagonal Elements: 
           Show misclassifications. Lower values are better.
        
        3. Row Sums: 
           Total actual instances for each class.
        
        4. Column Sums: 
           Total predicted instances for each class.
        
        5. Color Intensity: 
           Darker blues indicate higher values.
        
        6. Normalized vs. Raw: 
           - Raw: Shows actual counts
           - Normalized: Shows proportions (sum to 1 for each row)
        
        7. Common Patterns:
           - Strong Diagonal: Good overall performance
           - Weak Diagonal: Poor overall performance
           - Dark Off-Diagonal: Common misclassification
        
        8. Class Imbalance: 
           Look for significantly different row sums.
        
        9. Precision vs. Recall:
           - Precision: Look down columns (few false positives)
           - Recall: Look across rows (few false negatives)
        
        10. Overall Accuracy:
            Sum of diagonal elements divided by total predictions.
        """
        ax_guide.text(0, 1, guide_text, va='top', ha='left', fontsize=12, 
                      bbox=dict(boxstyle='round,pad=0.5', facecolor='wheat', alpha=0.3),
                      fontweight='medium')
        
        plt.suptitle(f'Confusion Matrix: MoViNet-{model_class}-Stream ({ds_type})\n'
                     f'Accuracy: {accuracy*100:.4f}%', fontsize=22, fontweight='bold', y=1.02)
        
        plt.tight_layout()
        fname = f'confusion_matrix_{model_class}_{ds_type}_accuracy_{accuracy*100:.4f}.png'
        plt.savefig(fname, bbox_inches='tight', dpi=600)
        plt.close()
        print(f"Enhanced confusion matrix saved as {fname}")
        
        # Generate classification report
        report = classification_report(actual, predicted, target_names=labels, output_dict=True)
        report_df = pd.DataFrame(report).transpose()
        report_fname = f'classification_report_{model_class}_{ds_type}_at_training_split_{100-split}_{split}.csv'
        report_df.to_csv(report_fname)
        print(f"Classification report saved as {report_fname}")
        
    except Exception as e:
        print(f"Error in plotting confusion matrix: {str(e)}")
        import traceback
        traceback.print_exc()

In [19]:
def plot_confusion_matrix2(actual, predicted, labels, ds_type,model_class,accuracy,split):
  cm = tf.math.confusion_matrix(actual, predicted)
  split = 100 * split
  ax = sns.heatmap(cm, annot=True, fmt='g')
  sns.set(rc={'figure.figsize':(6, 16)})
  sns.set(font_scale=1.4)
  plt.title(f'Confusion Matrix: MoViNet-{model_class}-Stream ({ds_type})\n'
                  f'Accuracy: {accuracy*100:.4f}',
                  fontsize=16, pad=16)
  ax.set_xlabel('Predicted Action')
  ax.set_ylabel('Actual Action')
  plt.xticks(rotation=90)
  plt.yticks(rotation=0)
  ax.xaxis.set_ticklabels(labels)
  ax.yaxis.set_ticklabels(labels)
  plt.tight_layout()
  fname = f'confusion_matrix_alternative_presentation_{model_class}_{ds_type}_accuracy_{accuracy*100:.4f}%_at_training_split_{100-split}_{split}.png'
  plt.savefig(fname, bbox_inches='tight', dpi=300)
  plt.close()

In [20]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

def plot_training_metrics(results, model_id, initial_lr, batch_size, optimizer, resolution, split, accuracy):
    """
    Plot training and validation metrics (accuracy, loss, and learning rate) over epochs with improved visuals for print.
    
    Args:
    results (keras.callbacks.History): History object returned by model.fit().
    model_id (str): Identifier for the model.
    initial_lr (float): Initial learning rate.
    batch_size (int): Batch size used for training.
    optimizer (str): Name of the optimizer used.
    resolution (int): Input resolution for the model.
    split (float): Train-validation split ratio.
    accuracy (float): Test accuracy of the model.
    
    Returns:
    None
    """
    history = results.history  # Extract the history dictionary from the History object
    split = 100 * split
    plt.style.use('seaborn')
    fig = plt.figure(figsize=(20, 30))
    gs = GridSpec(4, 2, figure=fig, height_ratios=[1, 1, 1, 0.5])
    
    # Colors
    colors = {'train': '#C71585',  # Deep pink
              'val': '#696969',    # Dim gray
              'lr': '#FFA500'}     # Orange
    
    # Metrics
    metrics = ['accuracy', 'loss']
    
    for i, metric in enumerate(metrics):
        ax = fig.add_subplot(gs[i, :])
        epochs = range(1, len(history[metric]) + 1)
        
        # Training data
        ax.plot(epochs, history[metric], color=colors['train'], label=f'Training {metric.capitalize()}', linewidth=2, marker='o')
        
        # Validation data
        ax.plot(epochs, history[f'val_{metric}'], color=colors['val'], label=f'Validation {metric.capitalize()}', linewidth=2, marker='o')
        
        # Add annotations for best/worst points
        best_epoch = np.argmax(history[f'val_{metric}']) if metric == 'accuracy' else np.argmin(history[f'val_{metric}'])
        best_value = history[f'val_{metric}'][best_epoch]
        ax.annotate(f'Best: {best_value:.4f}', xy=(best_epoch+1, best_value), xytext=(5, 5), 
                    textcoords='offset points', ha='left', va='bottom',
                    bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
        
        ax.set_title(f'{metric.capitalize()} Over Epochs', fontsize=16)
        ax.set_xlabel('Epoch', fontsize=12)
        ax.set_ylabel(metric.capitalize(), fontsize=12)
        ax.legend(fontsize=10)
        ax.grid(True, linestyle='--', alpha=0.7)
    
    # Learning Rate Over Epochs
    ax = fig.add_subplot(gs[2, :])
    if 'lr' in history:
        ax.plot(epochs, history['lr'], color=colors['lr'], linewidth=2, marker='o')
        ax.set_yscale('log')
        ax.set_title('Learning Rate Over Epochs', fontsize=16)
        ax.set_xlabel('Epoch', fontsize=12)
        ax.set_ylabel('Learning Rate', fontsize=12)
        ax.grid(True, linestyle='--', alpha=0.7)
        
        # Annotate initial and final learning rates
        ax.annotate(f'Initial LR: {history["lr"][0]:.2e}', xy=(1, history["lr"][0]), xytext=(5, 5), 
                    textcoords='offset points', ha='left', va='bottom',
                    bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
        ax.annotate(f'Final LR: {history["lr"][-1]:.2e}', xy=(len(epochs), history["lr"][-1]), xytext=(5, 5), 
                    textcoords='offset points', ha='right', va='top',
                    bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
                    arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
    else:
        ax.text(0.5, 0.5, 'Learning Rate data not available', ha='center', va='center', fontsize=16)
    
    # Info text
    info_text = [f"Model: {model_id}",
                 f"Initial Learning Rate: {initial_lr}",
                 f"Final Learning Rate: {history['lr'][-1]:.2e}" if 'lr' in history else "Final Learning Rate: Not available",
                 f"Batch Size: {batch_size}",
                 f"Optimizer: {optimizer}",
                 f"Resolution: {resolution}",
                 f"Total Epochs: {len(history['accuracy'])}",
                 f"Early Stopping: {'Yes' if len(history['accuracy']) < 30 else 'No'}",
                 f"Train-Val Split: {100 - split:.2f}/{split:.2f}"]

    # Summary text
    summary_text = [f"Test Accuracy: {accuracy*100:.2f}%",
                    f"Best Val Accuracy: {max(history['val_accuracy'])*100:.2f}% (Epoch {np.argmax(history['val_accuracy'])+1})",
                    f"Lowest Val Loss: {min(history['val_loss']):.4f} (Epoch {np.argmin(history['val_loss'])+1})",
                    f"Overfitting: {'Not observed' if max(history['accuracy']) - max(history['val_accuracy']) < 0.05 else 'Observed'}"]

    # Add info and summary as text
    ax_info = fig.add_subplot(gs[3, 0])
    ax_info.axis('off')
    ax_info.text(0, 1, '\n'.join(info_text), verticalalignment='top', fontsize=12)
    ax_info.set_title('Model Information', fontsize=16)

    ax_summary = fig.add_subplot(gs[3, 1])
    ax_summary.axis('off')
    ax_summary.text(0, 1, '\n'.join(summary_text), verticalalignment='top', fontsize=12)
    ax_summary.set_title('Performance Summary', fontsize=16)

    # Overall title
    fig.suptitle(f"Training Metrics for MoviNet {model_id} Stream at {100 - split}%-{split}% train test split", fontsize=20, y=0.95)

    # Adjust layout and save
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(f"training_metrics_for_{model_id}_at_train-test_split_of_{split}.png", dpi=300, bbox_inches='tight')
    plt.close()


In [21]:
import json
from datetime import datetime

def export_model_variants(model, model_id, resolution, num_frames, accuracy, split, train_dict, model_weights):
    """
    Export the model in various formats for single-batch and multi-batch scenarios,
    as well as multi-frame scenarios.
    
    Args:
    model: The TensorFlow model to export
    model_id (str): Identifier for the model
    resolution (int): Input resolution for the model
    accuracy (float): Accuracy of the model
    """
    exported_models = {}
    split = 100 * split

    # Calculate initial model size and parameters
    initial_model_size = get_model_size(model_weights)
    initial_params = model.count_params()

    # Determine initial numerical operations
    initial_num_ops = get_numerical_operations(model)

    for batch_size, batch_type in zip([1, 2], ['single', 'dual']):
        saved_model_dir = f'saved_model_{batch_type}_batch_{num_frames}_frames'
        
        input_shape = [batch_size, num_frames, resolution, resolution, 3]
        
        export_saved_model.export_saved_model(
            model=model,
            input_shape=input_shape,
            export_path=saved_model_dir,
            causal=True,
            bundle_input_init_states_fn = False 
        )
        converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
        # Optimization configuration
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        # FP16 conversion
        converter.target_spec.supported_types = [tf.float16]
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops
            tf.lite.OpsSet.SELECT_TF_OPS     # Enable select TensorFlow ops
        ]
        converter.experimental_new_converter = True

        try:
            tflite_model = converter.convert()
            print(f"Successfully converted {model_id} to TFLite")
            timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
            # Save with comprehensive filename
            fp16_filename = (
                f"movinet_{model_id}_with_"
                f"_train-size_{100 - split} and test-size_{split}_"
                f"test accuracy{accuracy*100:.2f}_"
                f"batch {batch_type}_"
                f"number of frames at a time {num_frames}_"
                f"at {timestamp}_ fp16_with_frame resolution and bundle_input_init_states_fn was set to False"
                f"{resolution}.tflite"
            )
            
            with open(fp16_filename, 'wb') as f:
                f.write(tflite_model)
                print(f"Saved TFLite model to {fp16_filename}")
                
        except Exception as e:
            print(f"Conversion failed for {model_id}: {str(e)}")
            continue
  
        exported_model_size = os.path.getsize(fp16_filename)
        
        exported_models[f'fp16_{batch_type}batch_{num_frames}frames'] = {
            'filename': fp16_filename,
            'input_shape': input_shape,
            'initial_model_size': initial_model_size,
            'exported_model_size': exported_model_size,
            'initial_num_ops': initial_num_ops,
            'exported_num_ops': 'fp16',
            'total_parameters': initial_params
        }

    # Save information to JSON file
    json_filename = f'model_{model_id}_export_info.json'
    with open(json_filename, 'w') as f:
        json.dump(exported_models, f, indent=4)

    return exported_models

def get_model_size(model_weights):
    
    size = os.path.getsize(model_weights)
    return size

def get_numerical_operations(model):
    # Determine the numerical operations used in the model
    if tf.keras.mixed_precision.global_policy().name == 'mixed_float16':
        return 'mixed_float16'
    else:
        return 'float32'

In [22]:

def setup_and_train_movinet_models(models_dict):
    for model_id, params in models_dict.items():
        batch_size = params['batch_size']
        num_frames = params['num_frames']
        resolution = params['resolution']
        split = params['split']

        print(f"Setting up MoViNet-{model_id} with resolution {resolution}\n")
        
        train_dict, test_dict = train_test_split(t, test_split=split)
        per_model(t, train_dict, test_dict, model_id, split)
        
        train_generator = FrameGenerator(train_dict, num_frames, resolution, training=True)
        test_generator = FrameGenerator(test_dict, num_frames, resolution, training=False)

        train_ds = prepare_dataset(train_generator, batch_size)
        test_ds = prepare_dataset(test_generator, batch_size)

        for frames, labels in train_ds.take(1):
              print(f"Shape: {frames.shape}")
              print(f"Label: {labels.shape}")
         
        
        use_positional_encoding = model_id in {'A3', 'A4', 'A5'}

        backbone = movinet.Movinet(
            model_id=model_id.lower(),
            causal=True,
            conv_type='2plus1d',
            se_type='2plus3d',
            activation='hard_swish',
            gating_activation='hard_sigmoid',
            use_positional_encoding=use_positional_encoding,
            use_external_states=False
        )

        backbone.Trainable = False

        model = movinet_model.MovinetClassifier(
            backbone,
            num_classes=600,
            output_states=True
        )

        # Load pre-trained weights
        url = f"https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_{model_id.lower()}_stream.tar.gz"
        tar_filename = f"movinet_{model_id.lower()}_stream.tar.gz"
        checkpoint_dir = f"movinet_{model_id.lower()}_stream"

        os.system(f"wget {url} -O {tar_filename} -q")
        os.system(f"tar -xvf {tar_filename}")

        checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
        checkpoint = tf.train.Checkpoint(model=model)
        status = checkpoint.restore(checkpoint_path)
        status.assert_existing_objects_matched()
        print(f"\nLoaded weights for MoViNet-{model_id} from {checkpoint_path}")

        model = build_classifier(batch_size, num_frames, resolution, backbone, 8)
        
        loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        model.compile(loss=loss_obj, optimizer='adam', metrics=['accuracy'])
        lr_scheduler = CustomLearningRateScheduler(lr_schedule)

        callbacks = [
            CustomLearningRateLogger(),
            CustomModelCheckpoint(
                filepath="trained_model.weights.h5",
                save_weights_only=True,
                verbose=1
            ),
            CustomReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=10,
                min_lr=1e-10,
                verbose=1,
                cooldown=1,
                min_delta=1e-3
            ),
            CustomEarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True,
                verbose=1
            ),
            lr_scheduler,
            CustomTensorBoard(log_dir='./logs')
        ]

        results = model.fit(
            train_ds,
            validation_data=test_ds,
            epochs=30,
            callbacks=callbacks,
            verbose=1
            # class_weight=class_weights
        )

        # Evaluate the model
        evaluation_results = model.evaluate(test_ds,return_dict=True)
        accuracy = evaluation_results['accuracy']
    

        # Plot training metrics
        plot_training_metrics(results, model_id, 0.001, batch_size, 'adam', resolution, split, accuracy)
       
        # Get actual and predicted labels
        actual, predicted = get_actual_predicted_labels(test_ds, model)
        label_names = list(test_generator.class_ids_for_name.keys())
                
        plot_confusion_matrix(actual, predicted, label_names, 'test', model_id,accuracy,split)
        plot_confusion_matrix2(actual, predicted, label_names, 'test', model_id,accuracy,split)
    
        print(f'\nAccuracy: {accuracy:.4f}')     
        
        model_id = model_id.lower()
        use_positional_encoding = model_id in {'a3', 'a4', 'a5'}
        resolution = resolution

        # Create backbone and model.
        backbone = movinet.Movinet(
            model_id=model_id,
            causal=True,
            conv_type='2plus1d',
            se_type='2plus3d',
            activation='hard_swish',
            gating_activation='hard_sigmoid',
            use_positional_encoding=use_positional_encoding,
            use_external_states=True,
        )

        model = movinet_model.MovinetClassifier(
                backbone,
                num_classes=8,
                output_states=True)

        model.load_weights('/kaggle/working/trained_model.weights.h5')
        # Export model variants
        exported_models = export_model_variants(model, model_id, resolution, num_frames, accuracy, split, train_dict, '/kaggle/working/trained_model.weights.h5')
        # export_model_variants(model, model_id, resolution, 1, accuracy, split, train_dict, '/kaggle/working/trained_model.weights.h5')
setup_and_train_movinet_models(models)

Setting up MoViNet-A3 with resolution 256

Plot saved: train_test_split_A3_split_70_30.png
Shape: (8, 8, 256, 256, 3)
Label: (8,)

Loaded weights for MoViNet-A3 from movinet_a3_stream/ckpt-1
Epoch 1/30
    239/Unknown - 536s 2s/step - loss: 1.0961 - accuracy: 0.6145
Epoch 1: saving model to trained_model.weights.h5
239/239 [==============================] - 700s 2s/step - loss: 1.0961 - accuracy: 0.6145 - val_loss: 1.7346 - val_accuracy: 0.6546 - lr: 0.0010 - learning_rate: 0.0010
Epoch 2/30
239/239 [==============================] - ETA: 0s - loss: 0.5550 - accuracy: 0.8117
Epoch 2: saving model to trained_model.weights.h5
239/239 [==============================] - 536s 2s/step - loss: 0.5550 - accuracy: 0.8117 - val_loss: 0.6449 - val_accuracy: 0.8056 - lr: 0.0010 - learning_rate: 0.0010
Epoch 3/30
239/239 [==============================] - ETA: 0s - loss: 0.3107 - accuracy: 0.9012
Epoch 3: saving model to trained_model.weights.h5
239/239 [==============================] - 536s 2s/st

<ipython-input-20-b777c728350a>:23: MatplotlibDeprecationWarning:

The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.



1/1 [==============================] - 0s 137ms/step
Enhanced confusion matrix saved as confusion_matrix_A3_test_accuracy_97.4638.png
Classification report saved as classification_report_A3_test_at_training_split_70.0_30.0.csv

Accuracy: 0.9746
Successfully converted a3 to TFLite
Saved TFLite model to movinet_a3_with__train-size_70.0 and test-size_30.0_test accuracy97.46_batch single_number of frames at a time 8_at 20250303-162737_ fp16_with_frame resolution and bundle_input_init_states_fn was set to False256.tflite
Successfully converted a3 to TFLite
Saved TFLite model to movinet_a3_with__train-size_70.0 and test-size_30.0_test accuracy97.46_batch dual_number of frames at a time 8_at 20250303-163408_ fp16_with_frame resolution and bundle_input_init_states_fn was set to False256.tflite
